In [1]:
import os
from os.path import expanduser
import socket
import sys
import glob

# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "pyfesom")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"pyfesom")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules") # add custom Vibe 's modules
#sys.path.append("pyfesom") # add standard 's modules
#########################################

computer =  bfn1001
working directory set to /home/hbkoziel/pyfesom


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
import pyfesom as pf
#from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
import cmocean as cmaps
#from matplotlib.colors import ListedColormap

In [3]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = cmaps.cm.curl
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1990
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = False


########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/pyfesom/figures/Arc12/ already existing


In [4]:
mesh = pf.load_mesh(meshpath, usepickle=True,get3d=get3d)
#mesh = fesom_mesh(meshpath, get3d = get3d)
#mesh = load_mesh(meshpath)

/home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh


In [5]:
mesh


FESOM mesh:
path                  = /gfs2/work/hbkvsk12/mesh/meshArc4.5
alpha, beta, gamma    = 50, 15, -90
number of 2d nodes    = 753179
number of 2d elements = 1480268
number of 3d nodes    = 16950142

        

In [6]:
# Loading data from CSV Files
print 'loading...'
list=glob.glob(outputpath+'PARmld*_calctrend.csv')

if len(list) > 0:
    print 'trends already calculated'
    APR = genfromtxt(outputpath+'PARmld_APR_1990_2015_calctrend.csv', delimiter=';')
    print 'April...'
    MAY = genfromtxt(outputpath+'PARmld_MAY_1990_2015_calctrend.csv', delimiter=';')
    print 'May...'
    JUN = genfromtxt(outputpath+'PARmld_JUN_1990_2015_calctrend.csv', delimiter=';')
    print 'June...'
    JUL = genfromtxt(outputpath+'PARmld_JUL_1990_2015_calctrend.csv', delimiter=';')
    print 'July...'
    AUG = genfromtxt(outputpath+'PARmld_AUG_1990_2015_calctrend.csv', delimiter=';')
    print 'August...'
    SEP = genfromtxt(outputpath+'PARmld_SEP_1990_2015_calctrend.csv', delimiter=';')
    print 'September...'
else:
    print 'and calculating trend from yearly data'
    X = genfromtxt(outputpath+'PARmld_APR_1990_2015_trend.csv', delimiter=';')
    result_fit = np.ma.polyfit(years,X,1)
    APR = result_fit[0,:]
    del X
    del result_fit
    np.savetxt(outputpath+'PARmld_APR_1990_2015_calctrend.csv', APR, delimiter=';')
    print 'April...'

    X = genfromtxt(outputpath+'PARmld_MAY_1990_2015_trend.csv', delimiter=';')
    result_fit = np.ma.polyfit(years,X,1)
    MAY = result_fit[0,:]
    del X
    del result_fit
    np.savetxt(outputpath+'PARmld_MAY_1990_2015_calctrend.csv', MAY, delimiter=';')
    print 'May...'

    X = genfromtxt(outputpath+'PARmld_JUN_1990_2015_trend.csv', delimiter=';')
    result_fit = np.ma.polyfit(years,X,1)
    JUN = result_fit[0,:]
    del X
    del result_fit
    np.savetxt(outputpath+'PARmld_JUN_1990_2015_calctrend.csv', JUN, delimiter=';')
    print 'June...'

    X = genfromtxt(outputpath+'PARmld_JUL_1990_2015_trend.csv', delimiter=';')
    result_fit = np.ma.polyfit(years,X,1)
    JUL = result_fit[0,:]
    del X
    del result_fit
    np.savetxt(outputpath+'PARmld_JUL_1990_2015_calctrend.csv', JUL, delimiter=';')
    print 'July...'

    X = genfromtxt(outputpath+'PARmld_AUG_1990_2015_trend.csv', delimiter=';')
    result_fit = np.ma.polyfit(years,X,1)
    AUG = result_fit[0,:]
    del X
    del result_fit
    np.savetxt(outputpath+'PARmld_AUG_1990_2015_calctrend.csv', AUG, delimiter=';')
    print 'August...'

    X = genfromtxt(outputpath+'PARmld_SEP_1990_2015_trend.csv', delimiter=';')
    result_fit = np.ma.polyfit(years,X,1)
    SEP = result_fit[0,:]
    del X
    del result_fit
    np.savetxt(outputpath+'PARmld_SEP_1990_2015_calctrend.csv', SEP, delimiter=';')
    print 'September...'

loading...
and calculating trend from yearly data
April...


IOError: /scratch/usr/hbkoziel/Arc12/PARmld_MAY_1990_2015_trend.csv not found.

In [ ]:
SEP.mean()
SEP.min()
SEP.max()

In [ ]:
contours = [-2, 2, 0.1]
contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
#contours=np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
#fig.suptitle('Density (kg m$^{-3}$)')
fig.subplots_adjust(wspace=0.02,hspace=0.02)
elem2=mesh.elem#[mesh.no_cyclic_elem,:]

ax1 = plt.subplot(3,2,1)
data2=APR
d=data2[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
#map.drawcoastlines()
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
plt.annotate('Apr.', xy=(0.75, 0.9), xycoords='axes fraction', fontsize = 20,
             fontweight = 'bold', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

ax2 = plt.subplot(3,2,2)
data2=MAY
d=data2[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
#map.drawcoastlines()
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
plt.annotate('May.', xy=(0.75, 0.9), xycoords='axes fraction', fontsize = 20,
             fontweight = 'bold', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

ax3 = plt.subplot(3,2,3)
data2=JUN
d=data2[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
#map.drawcoastlines()
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
plt.annotate('Jun.', xy=(0.75, 0.9), xycoords='axes fraction', fontsize = 20,
             fontweight = 'bold', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


ax4 = plt.subplot(3,2,4)
data2=JUL
d=data2[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
#map.drawcoastlines()
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
plt.annotate('Jul.', xy=(0.75, 0.9), xycoords='axes fraction', fontsize = 20,
             fontweight = 'bold', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


ax5 = plt.subplot(3,2,5)
data2=AUG
d=data2[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
#map.drawcoastlines()
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
plt.annotate('Aug.', xy=(0.75, 0.9), xycoords='axes fraction', fontsize = 20,
             fontweight = 'bold', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


ax6 = plt.subplot(3,2,6)
data2=SEP
d=data2[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
#map.drawcoastlines()
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
plt.annotate('Sep.', xy=(0.75, 0.9), xycoords='axes fraction', fontsize = 20,
             fontweight = 'bold', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.14, 0.1, 0.75, 0.02])
cb = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cb.set_label(label='$[PAR_{mld}$ (W m$^{-2}$ yr$^{-1}$)', size='large')
cb.ax.tick_params(labelsize='large')

if export_plot == True:
    plt.savefig(savepath+'trend/PARmld_trend_'+str(first_year)+'_'+str(last_year)+'.png', dpi = dpicnt, bbox_inches='tight') 
        